## How do I make an RDD?

RDDs can be created from stable storage or by transforming other RDDs. Run the cells below to create RDDs from files on the local drive.  All data files can be downloaded from https://www.cse.ust.hk/msbd5003/data/
For example, https://www.cse.ust.hk/msbd5003/data/fruits.txt

In [27]:
# Read data from local file system:
print(sc.version)

fruits = sc.textFile('../data/fruits.txt')
yellowThings = sc.textFile('../data/yellowthings.txt')
print(fruits.collect())
print(yellowThings.collect())

3.0.0
['apple', 'banana', 'canary melon', 'grap', 'lemon', 'orange', 'pineapple', 'strawberry']
['banana', 'bee', 'butter', 'canary melon', 'gold', 'lemon', 'pineapple', 'sunflower']


In [ ]:
# Read data from HDFS :
fruits = sc.textFile('hdfs://url:9000/pathname/fruits.txt')
fruits.collect()

----------

##  RDD operations

In [3]:
# map
fruitsReversed = fruits.map(lambda fruit: fruit[::-1])

In [12]:
# fruitsReversed = fruits.map(lambda fruit: fruit[::-1])
fruitsReversed.persist()
# try changing the file and re-execute with and without cache
print(fruitsReversed.collect())
# What happens when you uncomment the first line and run the whole program again with cache()?

['elppab', 'ananab', 'nolem yranac', 'parg', 'nomel', 'egnaro', 'elppaenip', 'yrrebwarts']


In [13]:
# filter
shortFruits = fruits.filter(lambda fruit: len(fruit) <= 5)
print(shortFruits.collect())

['grap', 'lemon']


In [14]:
# flatMap
characters = fruits.flatMap(lambda fruit: list(fruit))
print(characters.collect())

['b', 'a', 'p', 'p', 'l', 'e', 'b', 'a', 'n', 'a', 'n', 'a', 'c', 'a', 'n', 'a', 'r', 'y', ' ', 'm', 'e', 'l', 'o', 'n', 'g', 'r', 'a', 'p', 'l', 'e', 'm', 'o', 'n', 'o', 'r', 'a', 'n', 'g', 'e', 'p', 'i', 'n', 'e', 'a', 'p', 'p', 'l', 'e', 's', 't', 'r', 'a', 'w', 'b', 'e', 'r', 'r', 'y']


In [15]:
# union
fruitsAndYellowThings = fruits.union(yellowThings)
print(fruitsAndYellowThings.collect())

['bapple', 'banana', 'canary melon', 'grap', 'lemon', 'orange', 'pineapple', 'strawberry', 'banana', 'bee', 'butter', 'canary melon', 'gold', 'lemon', 'pineapple', 'sunflower']


In [16]:
# intersection
yellowFruits = fruits.intersection(yellowThings)
print(yellowFruits.collect())

['pineapple', 'canary melon', 'lemon', 'banana']


In [17]:
# distinct
distinctFruitsAndYellowThings = fruitsAndYellowThings.distinct()
print(distinctFruitsAndYellowThings.collect())

['orange', 'pineapple', 'canary melon', 'lemon', 'bee', 'banana', 'butter', 'gold', 'sunflower', 'apple', 'grap', 'strawberry']


### RDD actions
Following are examples of some of the common actions available. For a detailed list, see [RDD Actions](https://spark.apache.org/docs/2.0.0/programming-guide.html#actions).

Run some transformations below to understand this better. Place the cursor in the cell and press **SHIFT + ENTER**.

In [18]:
# collect
fruitsArray = fruits.collect()
yellowThingsArray = yellowThings.collect()
print(fruitsArray)

['apple', 'banana', 'canary melon', 'grap', 'lemon', 'orange', 'pineapple', 'strawberry']


In [19]:
# count
numFruits = fruits.count()
print(numFruits)

8


In [20]:
# take
first3Fruits = fruits.take(3)
print(first3Fruits)

['apple', 'banana', 'canary melon']


In [21]:
# reduce
letterSet = fruits.map(lambda fruit: set(fruit)).reduce(lambda x, y: x.union(y))
print(letterSet)

{'o', 'r', 'a', 'i', 'p', 'g', 'c', ' ', 'l', 'y', 'e', 'w', 'n', 'b', 'm', 't', 's'}


In [22]:
letterSet = fruits.flatMap(lambda fruit: list(fruit)).distinct().collect()
print(letterSet)

['p', 'l', 'b', 'c', 'r', 'y', 'g', 'i', 's', 'a', 'e', 'n', ' ', 'm', 'o', 't', 'w']


### Closure

In [23]:
counter = 0
rdd = sc.parallelize(range(10))

# Wrong: Don't do this!!
def increment_counter(x):
    global counter
    counter += x

print(rdd.collect())
rdd.foreach(increment_counter)

print(counter)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0


In [29]:
rdd = sc.parallelize(range(10))
accum = sc.accumulator(0)

def g(x):
    global accum
    accum += x

a = rdd.foreach(g)

print(accum.value)

-45


In [36]:
rdd = sc.parallelize(range(10))
accum = sc.accumulator(0)

def g(x):
    global accum
    accum += x
    return x * x

a = rdd.map(g)
print(accum.value)
#print(a.reduce(lambda x, y: x+y))
a.cache()
tmp = a.count()
print(accum.value)
print(rdd.reduce(lambda x, y: x+y))

tmp = a.count()
print(accum.value)
print(rdd.reduce(lambda x, y: x+y))


0
45
45
45
45


### Computing Pi using Monte Carlo simulation

In [3]:
# From the official spark examples.

import random
import time

partitions = 1000
n = 1000 * partitions

def f(_):
    x = random.random()
    y = random.random()
    return 1 if x ** 2 + y ** 2 < 1 else 0

count = sc.parallelize(range(1, n + 1), partitions) \
          .map(f).sum()

print("Pi is roughly", 4.0 * count / n)

Pi is roughly 3.140944


In [26]:
# Example: glom
import sys
import random

def f(_):
    return random.random()

a = sc.parallelize(range(0,100),10)
print(a.collect())
print(a.glom().collect())
print(a.map(f).glom().collect())

# Weird behavior: Initially, random numbers are synched across all workers, but will get 
# out-of-sync after a large (e.g, 1000000) number of random numbers have been generated.

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [10, 11, 12, 13, 14, 15, 16, 17, 18, 19], [20, 21, 22, 23, 24, 25, 26, 27, 28, 29], [30, 31, 32, 33, 34, 35, 36, 37, 38, 39], [40, 41, 42, 43, 44, 45, 46, 47, 48, 49], [50, 51, 52, 53, 54, 55, 56, 57, 58, 59], [60, 61, 62, 63, 64, 65, 66, 67, 68, 69], [70, 71, 72, 73, 74, 75, 76, 77, 78, 79], [80, 81, 82, 83, 84, 85, 86, 87, 88, 89], [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]]
[[0.7666128939512263, 0.7666128939512263, 0.7666128939512263, 0.7666128939512263, 0.7666128939512263, 0.7666128939512263, 0.7666128939512263, 0.7666128939512263, 0.7666128939512263, 0.76661289395122

In [7]:
# Example: mapPartition and mapPartitionWithIndex
a = sc.parallelize(range(0,20),4)
print(a.glom().collect())

def f(it):
    s = 0
    for i in it:
        s += i
        yield s

print(a.mapPartitions(f).collect())

def f(index, it):
    s = index
    for i in it:
        s += i
        yield s

print(a.mapPartitionsWithIndex(f).collect())

[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17, 18, 19]]
[0, 1, 3, 6, 10, 5, 11, 18, 26, 35, 10, 21, 33, 46, 60, 15, 31, 48, 66, 85]
[0, 1, 3, 6, 10, 6, 12, 19, 27, 36, 12, 23, 35, 48, 62, 18, 34, 51, 69, 88]


In [4]:
# Correct version
import random
import time

partitions = 1000
n = 1000 * partitions

seed = time.time()

def f(index, it):
    random.seed(index + seed)
    for i in it:
        x = random.random()
        y = random.random()
        yield 1 if x ** 2 + y ** 2 < 1 else 0

count = sc.parallelize(range(1, n + 1), partitions) \
          .mapPartitionsWithIndex(f).sum()

print("Pi is roughly", 4.0 * count / n)

Pi is roughly 3.141832


### Closure and Persistence

In [8]:
# RDD variables are references
A = sc.parallelize(range(10))
B = A.map(lambda x: x*2)
A = B.map(lambda x: x+1)
A.take(10)

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]

In [16]:
# Linear-time selection

data = [34, 67, 21, 56, 47, 89, 12, 44, 74, 43, 26]
A = sc.parallelize(data,2)
k = 4

while True:
    x = A.first()
    A1 = A.filter(lambda z: z < x)
    A2 = A.filter(lambda z: z > x)
    A1.cache()
    A2.cache()
    mid = A1.count()
    if mid == k:
        print(x)
        break
    
    if k < mid:
        A = A1
    else:
        A = A2
        k = k - mid - 1

43


In [10]:
sorted(data)

[12, 21, 26, 34, 43, 44, 47, 56, 67, 74, 89]

In [18]:
A = sc.parallelize(range(10))

x = 5
B = A.filter(lambda z: z < x)
# B.cache()
print(B.count())
x = 3
print(B.count())

5
5


In [23]:
A = sc.parallelize(range(10))

x = 5
B = A.filter(lambda z: z < x)
# B.cache()
B.unpersist()
# print(B.take(10))
print(B.collect())

x = 3
#print(B.take(10))
print(B.collect())
# collect() doesn't always re-collect data - bad design!
# Always use take() instead of collect()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


### Key-Value Pairs

In [28]:
# reduceByKey
numFruitsByLength = fruits.map(lambda fruit: (len(fruit), 1)).reduceByKey(lambda x, y: x + y)
print(numFruitsByLength.take(10))

[(6, 2), (12, 1), (4, 1), (10, 1), (5, 2), (9, 1)]


In [29]:
from operator import add

lines = sc.textFile('../data/course.txt')
counts = lines.flatMap(lambda x: x.split()) \
              .map(lambda x: (x, 1)) \
              .reduceByKey(add)
print(counts.sortByKey().take(20))

[('Big', 1), ('Course', 2), ('Description', 1), ('Information', 1), ('Lecture', 1), ('This', 1), ('across', 1), ('amount', 1), ('and', 3), ('as', 1), ('both', 1), ('centers.', 1), ('cloud', 1), ('commodity', 1), ('computing', 1), ('course', 1), ('data', 4), ('emerge', 1), ('enabling', 1), ('even', 1)]


In [30]:
print(counts.sortBy(lambda x: x[1], False).take(20))

[('data', 4), ('of', 3), ('and', 3), ('Course', 2), ('in', 2), ('the', 2), ('Information', 1), ('systems,', 1), ('cloud', 1), ('parallel', 1), ('as', 1), ('mining', 1), ('massive', 1), ('amount', 1), ('even', 1), ('servers', 1), ('centers.', 1), ('both', 1), ('hands-on', 1), ('this', 1)]


In [32]:
# Join simple example

products = sc.parallelize([(1, "Apple"), (2, "Orange"), (3, "TV"), (5, "Computer")])
#trans = sc.parallelize([(1, 134, "OK"), (3, 34, "OK"), (5, 162, "Error"), (1, 135, "OK"), (2, 53, "OK"), (1, 45, "OK")])
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

print(products.join(trans).take(20))

[(1, ('Apple', (134, 'OK'))), (1, ('Apple', (135, 'OK'))), (1, ('Apple', (45, 'OK'))), (2, ('Orange', (53, 'OK'))), (3, ('TV', (34, 'OK'))), (5, ('Computer', (162, 'Error')))]


### K-means clustering

In [ ]:
import numpy as np

def parseVector(line):
    return np.array([float(x) for x in line.split()])

def closestPoint(p, centers):
    bestIndex = 0
    closest = float("+inf")
    for i in range(len(centers)):
        tempDist = np.sum((p - centers[i]) ** 2)
        if tempDist < closest:
            closest = tempDist
            bestIndex = i
    return bestIndex

# The data file can be downloaded at http://www.cse.ust.hk/msbd5003/data/kmeans_data.txt
lines = sc.textFile('../data/kmeans_data.txt', 5)  

# The data file can be downloaded at http://www.cse.ust.hk/msbd5003/data/kmeans_bigdata.txt
# lines = sc.textFile('../data/kmeans_bigdata.txt', 5)  
# lines is an RDD of strings
K = 3
convergeDist = 0.01  
# terminate algorithm when the total distance from old center to new centers is less than this value

data = lines.map(parseVector).cache() # data is an RDD of arrays

kCenters = data.takeSample(False, K, 1)  # intial centers as a list of arrays
tempDist = 1.0  # total distance from old centers to new centers

while tempDist > convergeDist:
    closest = data.map(lambda p: (closestPoint(p, kCenters), (p, 1)))
    # for each point in data, find its closest center
    # closest is an RDD of tuples (index of closest center, (point, 1))
        
    pointStats = closest.reduceByKey(lambda p1, p2: (p1[0] + p2[0], p1[1] + p2[1]))
    # pointStats is an RDD of tuples (index of center,
    # (array of sums of coordinates, total number of points assigned))
    
    newCenters = pointStats.map(lambda st: (st[0], st[1][0] / st[1][1])).collect()
    # compute the new centers
    
    tempDist = sum(np.sum((kCenters[i] - p) ** 2) for (i, p) in newCenters)
    # compute the total disctance from old centers to new centers
    
    for (i, p) in newCenters:
        kCenters[i] = p
        
print("Final centers: ", kCenters)


### PageRank

In [ ]:
import re
from operator import add

def computeContribs(urls, rank):
    # Calculates URL contributions to the rank of other URLs.
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)

def parseNeighbors(urls):
    # Parses a urls pair string into urls pair."""
    parts = urls.split(' ')
    return parts[0], parts[1]

# Loads in input file. It should be in format of:
#     URL         neighbor URL
#     URL         neighbor URL
#     URL         neighbor URL
#     ...

# The data file can be downloaded at http://www.cse.ust.hk/msbd5003/data/*
lines = sc.textFile("../data/pagerank_data.txt", 2)
# lines = sc.textFile("../data/dblp.in", 5)

numOfIterations = 10

# Loads all URLs from input file and initialize their neighbors. 
links = lines.map(lambda urls: parseNeighbors(urls)) \
             .groupByKey()

# Loads all URLs with other URL(s) link to from input file 
# and initialize ranks of them to one.
ranks = links.mapValues(lambda neighbors: 1.0)

# Calculates and updates URL ranks continuously using PageRank algorithm.
for iteration in range(numOfIterations):
    # Calculates URL contributions to the rank of other URLs.
    contribs = links.join(ranks) \
                    .flatMap(lambda url_urls_rank:
                             computeContribs(url_urls_rank[1][0],
                                             url_urls_rank[1][1]))
    # After the join, each element in the RDD is of the form
    # (url, (list of neighbor urls, rank))
    
    # Re-calculates URL ranks based on neighbor contributions.
    # ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)
    ranks = contribs.reduceByKey(add).map(lambda t: (t[0], t[1] * 0.85 + 0.15))

print(ranks.top(5, lambda x: x[1]))

### Join vs. Broadcast Variables

In [ ]:
products = sc.parallelize([(1, "Apple"), (2, "Orange"), (3, "TV"), (5, "Computer")])
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

print(trans.join(products).take(20))


In [ ]:
products = {1: "Apple", 2: "Orange", 3: "TV", 5: "Computer"}
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

broadcasted_products = sc.broadcast(products)

results = trans.map(lambda x: (x[0], broadcasted_products.value[x[0]], x[1]))
#  results = trans.map(lambda x: (x[0], products[x[0]], x[1]))
print(results.take(20))
